In [ ]:
# Imports

import pandas as pd
import os
import csv
import pandas as pd

# Sets iniciais de Dataset
O que eu preciso?
1) Um dataset que contém o id do usuário e um rating para um veículo, também com o ID dele
2) Um dataset que contém o id do veículo com suas características

In [ ]:
# Funções auxiliares

def ler_csv_para_dataframe(caminho_arquivo_csv):
    with open(caminho_arquivo_csv, 'r', newline='', encoding='utf-8') as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        linhas = list(leitor_csv)        
    df = pd.DataFrame(linhas[1:], columns=linhas[0])
    return df
    
def renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova):
    for nome_arquivo in os.listdir(pasta):
        caminho_arquivo_antigo = os.path.join(pasta, nome_arquivo)
        if palavra_antiga in nome_arquivo:
            novo_nome_arquivo = nome_arquivo.replace(palavra_antiga, palavra_nova)
            caminho_arquivo_novo = os.path.join(pasta, novo_nome_arquivo)
            os.rename(caminho_arquivo_antigo, caminho_arquivo_novo)
    print("Arquivos renomeados com sucesso!")

def obter_nomes_apos_substring(pasta):
    nomes_extraidos = []
    for nome_arquivo in os.listdir(pasta):
        if "Scraped_Car_Review_" in nome_arquivo:
            parte_nome = nome_arquivo.split("Scraped_Car_Review_")[1].replace(".csv", "")
            nomes_extraidos.append(parte_nome)
    return nomes_extraidos

In [ ]:
# Tratamento dos arquivos nas pastas

pasta = './Data/'
palavra_antiga = "Reviews"
palavra_nova = "Review"
renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)

palavra_antiga = "Scrapped"
palavra_nova = "Scraped"
renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)

In [ ]:
# Upload dos datasets

marcas_de_carro = obter_nomes_apos_substring("./Data/")
todos_df = []

for marca in marcas_de_carro:
    file_path = f'./Data/Scraped_Car_Review_{marca}.csv'
    df = ler_csv_para_dataframe(file_path)
    df = df.drop(["Review_Title", "Review", "Review_Date", ""], axis=1)
    todos_df.append(df)

df_final = pd.concat(todos_df, ignore_index=True)
df_comID = df_final.dropna()